In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pathvalidate import sanitize_filename

In [2]:
# define paths and import data
path = '~/OneDrive - SekII Zürich/Schulleitung/Umfragen/Belastungsstudie MNG/'
file1 = 'BBS1.xlsx'
file2 = 'BBS2.xlsx'
file3 = 'BBS3.xlsx'
file_cols = 'columns.xlsx'

cols = pd.read_excel(path+file_cols)
df1 = pd.read_excel(path+file1)
df2 = pd.read_excel(path+file2)
df34 = pd.read_excel(path+file3)

# define dataframes
cols_sel1 = cols[cols['col_include']==1]
cols_sel4 = cols[cols['col_include']==3]
cols_lst1 = cols_sel1['col_ind'].to_list()
cols_lst4 = cols_sel4['col_ind'].to_list()
df1 = df1.iloc[:, cols_lst1]
df2 = df2.iloc[:, cols_lst1]
df3 = df34.iloc[:, cols_lst1]
df4 = df34.iloc[:, cols_lst4]

df_ext = pd.concat([df3, df4], axis=1)
df = pd.concat([df1, df2, df_ext]).reset_index()

# clean data
df.loc[df['Einschätzung_Leistung']=='knapp ', 'Einschätzung_Leistung'] = 'knapp'
for column in ['familie_we', 'soziale_kontakte_we', 'sport_we', 'instrument_we', 'social_media_we', 'gamen_we', 'streamen_we', 'andere_hobbies_we', 'nebenjob_we',
               'familie_wt', 'soziale_kontakte_wt', 'sport_wt', 'instrument_wt', 'social_media_wt', 'gamen_wt', 'streamen_wt', 'andere_hobbies_wt', 'nebenjob_wt']:
        df[column] = df[column].str.replace('keine Zeit gehabt dafür', 'keine Zeit dafür gehabt')

df['ha_fächer_we'] = df['ha_fächer_we'].str[:-1].str.split(';')
df['ha_fächer_wt'] = df['ha_fächer_wt'].str[:-1].str.split(';')
df['pv_fächer_we'] = df['pv_fächer_we'].str[:-1].str.split(';')
df['pv_fächer_wt'] = df['pv_fächer_wt'].str[:-1].str.split(';')
df['bel_fächer_6m'] = df['bel_fächer_6m'].str[:-1].str.split(';')
df['erarbeiten_fächer'] = df['erarbeiten_fächer'].str[:-1].str.split(';')
df['beratung'] = df['beratung'].str[:-1].str.split(';')


herkunft = {'Vor dem Eintritt ins MNG habe ich ein Untergymnasium besucht.': 'UG',
            'Vor dem Eintritt ins MNG habe ich eine Sekundarschule besucht.': 'Sek',
            'Andere Herkunft (z.B. Hospitant etc.)': 'andere'}

df['Schulische Herkunft'] = df['Schulische Herkunft'].map(herkunft)

ziel = {'Ich möchte mit möglichst wenig Aufwand durchkommen.': 'wenig Aufwand', 
        'In Fächern, die mich interessieren, möchte ich gute Noten erreichen.': 'gute Noten in einzelnen Fächern',
        'Ich möchte in möglichst vielen Fächern gute Noten erreichen.': 'gute Noten in vielen Fächern',
        'Ich möchte in möglichst vielen Fächern sehr gute Noten erreichen.': 'sehr gute Noten in vielen Fächern'}

df['Ziel'] = df['Ziel'].map(ziel)

begruendung = {'ein schulinternes Angebot ist mir zu wenig anonym': 'zu wenig anonym', 
              'ich will mit niemandem über meine Probleme sprechen': 'will mit niemandem sprechen', 
              'ich weiss nicht, wie man sich anmeldet': 'Anmeldung unklar', 
              'ich bezweifle den Nutzen von Beratungsangeboten': 'Nutzen zweifelhaft', 
              'andere Gründe': 'andere Gründe'}

df['beratung_begründung_nein'] = df['beratung_begründung_nein'].map(begruendung)


degree = {1: '1 - gar nicht', 2: '2 - wenig', 3: '3 - mittel', 4: '4 - stark', 5: '5 - sehr stark'}

# df['zeitl_bel_7d'] = df['zeitl_bel_7d'].map(degree)
# df['mental_bel_7d'] = df['mental_bel_7d'].map(degree)

# define additional columns with numerical data instead of strings
time_str2flt = {'0 - 1 Stunde': 0.5, '1 - 2 Stunden': 1.5, '2 - 3 Stunden': 2.5, '3 - 4 Stunden': 3.5, '4 - 5 Stunden': 4.5, 
                '5 - 6 Stunden': 5.5, '6 - 7 Stunden': 6.5, 'mehr': 7.5, }

df['pv_zeit_we_flt'] = df['pv_zeit_we'].map(time_str2flt)
df['ha_zeit_we_flt'] = df['ha_zeit_we'].map(time_str2flt)
df['pv_zeit_wt_flt'] = df['pv_zeit_wt'].map(time_str2flt)
df['ha_zeit_wt_flt'] = df['ha_zeit_wt'].map(time_str2flt)

count_pres_str2flt = {'1-3': 2.0, '4-6': 5.0, '7-9': 8.0, '10 und mehr': 11.0}
freq_nh_str2flt = {'nein': 0.0, 'weniger als einmal pro Woche': 0.5, 'einmal pro Woche': 1.0, 'zweimal pro Woche': 2.0, 
                 'mehr als zweimal pro Woche': 3.0}
count_nh_str2flt = {'Keines': 0.0, '1 Fach': 1.0, '2 Fächer': 2.0, '3 Fächer': 3.0, 'mehr': 4.0}

df_ext['anz_vorträge_arbeiten_flt'] = df_ext['anz_vorträge_arbeiten'].map(count_pres_str2flt)
df_ext['nachhilfe_frequenz_flt'] = df_ext['nachhilfe_frequenz'].map(freq_nh_str2flt)
df_ext['nachhilfe_fächer_flt'] = df_ext['nachhilfe_fächer'].map(count_nh_str2flt)

# define labels for pie charts
orders = {'Klassen': ['1. Klasse', '2. Klasse', '3. Klasse'], 
          'Schwerpunktfach': ['Biologie/Chemie', 'Physik/Anwendungen der Mathematik'], 
          'Geschlecht': ['männlich', 'weiblich', 'andere Identität'], 
          'Leistung': ['knapp', 'durchschnittlich', 'gut', 'sehr gut'], 
          'Herkunft': ['UG', 'Sek', 'andere'], 
          'Ziel': ['wenig Aufwand', 'gute Noten in einzelnen Fächern', 'gute Noten in vielen Fächern', 'sehr gute Noten in vielen Fächern'],
          'Ferien': ['nie', 'selten', 'manchmal', 'oft'],
          'Beratungsangebot': ['Schulärztin', 'Schulsozialarbeiter', 'Beratungslehrperson', 'Klassenlehrperson'],
          'Beratung_genutzt': ['ja', 'nein'], 'Beratung_vorstellbar': ['ja', 'nein'],
          'Begründung_nein': ['zu wenig anonym', 'will mit niemandem sprechen', 'Anmeldung unklar', 'Nutzen zweifelhaft', 'andere Gründe'],
          'PV_Zeitaufwand_WE': ['0 - 1 Stunde', '1 - 2 Stunden', '2 - 3 Stunden', '3 - 4 Stunden', '4 - 5 Stunden', '5 - 6 Stunden', '6 - 7 Stunden', 'mehr'], 
          'HA_Zeitaufwand_WE': ['0 - 1 Stunde', '1 - 2 Stunden', '2 - 3 Stunden', '3 - 4 Stunden', '4 - 5 Stunden', 'mehr'],
          'PV_Zeitaufwand_WT': ['0 - 1 Stunde', '1 - 2 Stunden', '2 - 3 Stunden', '3 - 4 Stunden', '4 - 5 Stunden', '5 - 6 Stunden', '6 - 7 Stunden', 'mehr'],
          'HA_Zeitaufwand_WT': ['0 - 1 Stunde', '1 - 2 Stunden', '2 - 3 Stunden', '3 - 4 Stunden', '4 - 5 Stunden', '5 - 6 Stunden', '6 - 7 Stunden', 'mehr'],
          'Anzahl_Vorträge': ['1-3', '4-6', '7-9', '10 und mehr'],
          'NH_Frequenz': ['nein', 'weniger als einmal pro Woche', 'einmal pro Woche', 'zweimal pro Woche', 'mehr als zweimal pro Woche'],
          'NH_Anzahl': ['Keines', '1 Fach', '2 Fächer', '3 Fächer', 'mehr'],
          'PV_Fächer_WE': ['D', 'M', 'Inf', 'F/I', 'E', 'G', 'Gg', 'B', 'C', 'P', 'Mu', 'BG'],
          'HA_Fächer_WE': ['D', 'M', 'Inf', 'F/I', 'E', 'G', 'Gg', 'B', 'C', 'P', 'Mu', 'BG'],
          'PV_Fächer_WT': ['D', 'M', 'Inf', 'F/I', 'E', 'G', 'Gg', 'B', 'C', 'P', 'Mu', 'BG'],
          'HA_Fächer_WT': ['D', 'M', 'Inf', 'F/I', 'E', 'G', 'Gg', 'B', 'C', 'P', 'Mu', 'BG'],
          'Zeitliche_Belastung_Fächer': ['D', 'M', 'Inf', 'F/I', 'E', 'G', 'Gg', 'B', 'C', 'P', 'Mu', 'BG'],
          'ausserhalb': ['D', 'M', 'Inf', 'F/I', 'E', 'G', 'Gg', 'B', 'C', 'P', 'Mu', 'BG'],
          'Beratungsangebot': ['Schulärztin', 'Schulsozialarbeiter', 'Beratungslehrpersonen', 'Klassenlehrperson']}

stack_options = {'bis zu 2 Stunden': 'a (< 2 h)', '2 - 4 Stunden': 'b (2-4 h)', '4 - 6 Stunden': 'c (4-6 h)', '6 - 8 Stunden': 'd (6-8 h)', 'mehr als 8 Stunden': 'e (> 8 h)', 'keine Zeit dafür gehabt': 'f (keine Zeit)', 'mache ich nicht': 'g (mache ich nicht)',
                 '1 (gering)': '1 (gering)', '2': '2', '3': '3', '4 (gross)': '4 (gross)',
                 'nicht hilfreich': 'a nicht hilfreich', 'eher nicht hilfreich': 'b eher nicht hilfreich', 'eher hilfreich': 'c eher hilfreich', 'sehr hilfreich': 'd sehr hilfreich',
                 'gar nicht': 'a gar nicht', 'eher nicht': 'b gar nicht', 'eher ja': 'c eher ja', 'ja, sehr': 'd ja, sehr'}

cols['labels'] = cols['labels'].map(orders)

In [3]:
df_m = df[df['Geschlecht']=='männlich']
df_w = df[df['Geschlecht']=='weiblich']
df_a = df[df['Geschlecht']=='andere Identität']

df_1 = df[df['Klassenstufe']=='1. Klasse']
df_2 = df[df['Klassenstufe']=='2. Klasse']
df_3 = df[df['Klassenstufe']=='3. Klasse']

df_sg = df[df['Einschätzung_Leistung']=='sehr gut']
df_g = df[df['Einschätzung_Leistung']=='gut']
df_d = df[df['Einschätzung_Leistung']=='durchschnittlich']
df_k = df[df['Einschätzung_Leistung']=='knapp']

df_bc = df[df['Schwerpunktfach']=='Biologie/Chemie']
df_pa = df[df['Schwerpunktfach']=='Physik/Anwendungen der Mathematik']

In [4]:
# function to plot histograms
def plot_hist(data, bins, range, suptitle, title, color, path):
    fig = plt.figure(figsize=(8, 5), dpi=150)
    mean = data.mean()
    title = title + f' (⌀ = {mean:.2f})'
    data.hist(bins=bins, range=range, rwidth=0.8, color=color, ax=plt.gca())
    plt.suptitle(suptitle)
    plt.title(title)
    # plt.xticks(range(1,7))
    plt.grid(False)
    plt.tight_layout()
    plt.savefig(path)
    plt.close()

# function to plot bar charts
def plot_bar(data, order, suptitle, title, color, path):
    fig = plt.figure(figsize=(8, 5), dpi=150)
    data.value_counts().reindex(order).plot.bar(color=color, ax=plt.gca())
    plt.suptitle(suptitle)
    plt.title(title)
    # plt.xticks(range(1,7))
    plt.grid(False)
    plt.tight_layout()
    plt.savefig(path)
    plt.close()

# function to plot pie charts
def plot_pie(data, order, suptitle, title, path):
    fig = plt.figure(figsize=(8, 5), dpi=150)
    data.value_counts().reindex(order).plot.pie(autopct='%1.0f%%', ylabel='', ax=plt.gca())
    plt.suptitle(suptitle)
    plt.title(title)
    # plt.xticks(range(1,7))
    plt.grid(False)
    plt.tight_layout()
    plt.savefig(path)
    plt.close()

# function to plot stacked bar plot
def plot_stacked(data, suptitle, title, path):
    fig = plt.figure(figsize=(8, 5), dpi=150)
    ax = plt.subplot(111)
    data.plot.barh(stacked=True, ax=plt.gca())
    plt.suptitle(suptitle)
    plt.title(title)
    # plt.xticks(range(1,7))
    plt.grid(False)
    plt.tight_layout()
    # Shrink current axis by 20%
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.6, box.height])

    # Put a legend to the right of the current axis
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.2))

    plt.savefig(path)
    plt.close()


In [5]:
subjects = {'Bildnerisches Gestalten': 'BG', 'BildnerischesGestalten': 'BG', 'Biologie': 'B', 'Chemie': 'C', 'Deutsch': 'D', 
            'Englisch': 'E', 'Französisch/Italienisch': 'F/I', 'Französisch / Italienisch': 'F/I','Geografie': 'Gg', 'Geschichte': 'G', 'Informatik': 'Inf', 
            'Mathematik': 'M', 'Physik': 'P', 'Musik': 'Mu',
            'Schulärztin': 'Schulärztin', 'Schulsozialarbeiter': 'Schulsozialarbeiter', 
            'Beratungslehrpersonen': 'Beratungslehrpersonen', 'Klassenlehrperson': 'Klassenlehrperson'}

# dfs = [df]
# labels = ['alle']
# suffixes = ['_all']
# colors = ['Black']
dfs = [df, df_m, df_w, df_a, df_1, df_2, df_3, df_bc, df_pa, df_sg, df_g, df_d, df_k]
labels = ['alle', 'nur männlich', 'nur weiblich', 'nur andere Identität', '1. Klasse', '2. Klasse', '3. Klasse', 'Biologie/Chemie', 'Physik/AM', 'sehr gut', 'gut', 'durchschnittlich', 'knapp']
suffixes = ['_all', '_m', '_w', '_a', '_1', '_2', '_3', '_bc', '_pa', '_sg', '_g', '_d', '_k']
colors = ['Black', 'CornflowerBlue', 'LightPink', 'DarkSalmon', 'AntiqueWhite', 'BurlyWood', 'DarkGoldenRod', 'DarkSlateBlue', 'FireBrick', 'LightGreen', 'MediumSeaGreen', 'LightSlateGrey', 'PaleVioletRed']

stack_max = 9

for dfr, label, suffix, color in zip(dfs, labels, suffixes, colors):
    for index, row in cols.iterrows():
        if (row['col_include'] == 1.0) | (row['col_include'] == 3.0):
            column = row['col_name']
            suptitle = row['col_description']
            fn = sanitize_filename(column)
            path = 'figures/'+fn+suffix+'.png'

            if row['col_diag'] == 'bar':
                order = row['labels']
                plot_bar(dfr[column], order, suptitle, label, color, path)

            if row['col_diag'] == 'bar_exp':
                dfplot = dfr.explode(column)
                order = row['labels']
                data = dfplot[column].map(subjects)
                plot_bar(data, order, suptitle, label, color, path)

            if row['col_diag'] == 'hist':
                plot_hist(dfr[column], 5, (0.5, 5.5), suptitle, label, color, path)

            if row['col_diag'] == 'pie':
                if column == 'Schwerpunktfach' and (suffix == '_1' or suffix == '_2'):
                    pass
                else:
                    order = row['labels']
                    plot_pie(dfr[column], order, suptitle, label, path)

    for i in range(1, stack_max+1):
        # filter columns with stack # i
        cols_stack = cols[cols['col_stack'] == 'stack'+str(i)]
        stack_lst = cols_stack['col_name'].to_list()
        
        # create labels
        desc = cols_stack['col_description'].str.split(':')
        xlabels = []
        for _, v in desc.items():
            suptitle = v[0]
            xlabels.append(v[1][1:])

        fn = sanitize_filename(suptitle)
        path = 'figures/'+fn+suffix+'.png'


        # prepare plot data
        df_temp = dfr[stack_lst]

        for item in df_temp:
            df_temp[item] = df_temp[item].map(stack_options)

        columns = df_temp.columns.to_list()
        options = df_temp[columns[0]].dropna().unique().tolist()
        options.sort()
        
        d = {}
        for opt in options:
            d[opt] = [(df_temp[col].value_counts())[opt] for col in columns]
        df_stack = pd.DataFrame(d, index=xlabels)

        # plot
        plot_stacked(df_stack, suptitle, label, path)




/var/folders/zd/l7mhvtvd2kl599sl105gvq3h0000gn/T/ipykernel_46089/2274324704.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp[item] = df_temp[item].map(stack_options)
/var/folders/zd/l7mhvtvd2kl599sl105gvq3h0000gn/T/ipykernel_46089/2274324704.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp[item] = df_temp[item].map(stack_options)
/var/folders/zd/l7mhvtvd2kl599sl105gvq3h0000gn/T/ipykernel_46089/2274324704.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

KeyError: 'c (4-6 h)'

In [16]:
i = 2
# filter columns with stack # i
cols_stack = cols[cols['col_stack'] == 'stack'+str(i)]
stack_lst = cols_stack['col_name'].to_list()

# create labels
desc = cols_stack['col_description'].str.split(':')
xlabels = []
for _, v in desc.items():
    suptitle = v[0]
    xlabels.append(v[1][1:])

fn = sanitize_filename(suptitle)
path = 'figures/'+fn+suffix+'.png'


# prepare plot data
df_temp = dfr[stack_lst]

for item in df_temp:
    df_temp[item] = df_temp[item].map(stack_options)

columns = df_temp.columns.to_list()
options = df_temp[columns[0]].dropna().unique().tolist()
options.sort()

d = {}
for opt in options:
    d[opt] = [(df_temp[col].value_counts())[opt] for col in columns]
df_stack = pd.DataFrame(d, index=xlabels)

# plot
plot_stacked(df_stack, suptitle, label, path)


/var/folders/zd/l7mhvtvd2kl599sl105gvq3h0000gn/T/ipykernel_46089/2182407547.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp[item] = df_temp[item].map(stack_options)
/var/folders/zd/l7mhvtvd2kl599sl105gvq3h0000gn/T/ipykernel_46089/2182407547.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp[item] = df_temp[item].map(stack_options)
/var/folders/zd/l7mhvtvd2kl599sl105gvq3h0000gn/T/ipykernel_46089/2182407547.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

KeyError: 'c (4-6 h)'

In [15]:
df_stack

,a (< 2 h),b (2-4 h),c (4-6 h),d (6-8 h),e (> 8 h),f (keine Zeit),g (mache ich nicht)
Familie,208,170,65,57,72,80,24
andere soziale Kontakte,172,126,60,66,70,140,42
Sport,207,150,49,51,44,112,63
Instrument,176,28,14,11,8,140,299
Social Media,220,146,53,50,37,79,91
Gamen,158,99,41,30,31,151,166
Streamen,177,83,21,19,15,165,196
Andere Hobbies,168,77,19,27,20,187,178
Nebenjob,18,7,3,5,4,84,555


In [187]:
df_stack

,a nicht hilfreich,b eher nicht hilfreich,c eher hilfreich,total
Weniger Hausaufgaben,36,86,124,246
Leistungsdruck reduzieren,14,37,121,172
mehr soziale Anlässe,36,60,108,204
mehr Wahlfreiheit im Fächerangebot,27,74,98,199
mehr Informationen zu Stress und Belastung,103,98,72,273


In [14]:
i

2